1️⃣ RAG with the new LangChain API (no LangGraph yet)


✅ Stack

langchain

langchain-ollama

faiss

Ollama (llama3, nomic-embed-text)

##### StreamLit user interface

First I had to install ipykernal through following command:
```python
conda install -n rag-chatbot ipykernel --update-deps --force-reinstall
```


| Use case               | Best choice           |
| ---------------------- | --------------------- |
| Just learning / hobby  | Sentence-Transformers |
| RAG app, local dev     | Ollama                |
| Production SaaS        | Paid OpenAI key       |
| Offline / private data | Local models          |


In [ ]:
## This is for .env file if we use openAI API keys..
%load_ext dotenv
%dotenv

##### Download and Install Ollama from https://ollama.com/

A. Install packages

In [ ]:
!pip install -U langchain langchain-ollama langchain-community faiss-cpu

The ollama pip package ≠ Ollama itself
It’s just a Python client that talks to the Ollama server.

B. Load and split documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain_text_splitters import TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate)
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import (ChatOpenAI, OpenAIEmbeddings)
from langchain_chroma import Chroma



Loader_pdf = PyPDFLoader('1._Intro_to_AI_-_Course_notes.pdf')
Loader_pdf = TextLoader('Alan_Turing.txt')

doc_list   = Loader_pdf.load()



token_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", chunk_size=200, chunk_overlap = 40
)
doc_list_token_split = token_splitter.split_documents(doc_list)


#embedding = OpenAIEmbeddings(model='text-embedding-3-small')
#vectorstore = Chroma.from_documents(documents=doc_list_token_split, embedding=embedding, persist_directory="intro-to-ai")

#loader = TextLoader("data.txt")
#docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=40
)

chunks = splitter.split_documents(doc_list)



In [16]:
#!pip install -U langchain-ollama
!ollama pull llama3 
!ollama pull nomic-embed-text
#!ollama run llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕██████████████████▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕██████████████████▏  12 KB                         
pulling 8ab4849b038c: 100% ▕██████████████████▏  254 B                         
pulling 577073ffcc6c: 100% ▕██████████████████▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 970aa74c0a90:   0% ▕                  ▏ 761 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   0% ▕                  ▏ 1.3 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   1% ▕     

C. Embeddings

In [55]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

D. Vector store

In [68]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    chunks,
    embeddings
)

retriever = vectorstore.as_retriever(search_type = "mmr", search_kwargs={"k": 1,"lambda_mult":0.7})


len(vectorstore.index_to_docstore_id)



301

E. LLM (new API)

In [69]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="llama3",
    temperature=0
)


F. Prompt

In [70]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant.
Use the context below to answer the question.

Context:
{context}

Question:
{question}
""")

G. RAG chain (modern langchain)

In [71]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)

H. Invoke

In [72]:
print(rag_chain.invoke("What did Alan Turing do?"))

I'm not seeing any information about Alan Turing in the provided document context. The document appears to be related to an "Intro to AI" course and doesn't mention Alan Turing specifically. If you're looking for information on Alan Turing, I'd be happy to help with that! He was a British mathematician, computer scientist, and logician who made significant contributions to the development of computer science, artificial intelligence, and cryptography.


##### pdf file is provided as external info source for our RAG system

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable